### Calculate all motif

In [94]:
from itertools import combinations
from myf import calculate_g_xt, calculate_f_jk

def calculate_single_motif(profiles, gamma):
    min_distance = float('inf')
    best_pair = None

    for (i, dp1), (j, dp2) in combinations(enumerate(profiles), 2):
        distance = calculate_f_jk(dp1, dp2)
        if distance < min_distance and distance < gamma:
            min_distance = distance
            best_pair = (i, j)
    
    if best_pair is None:
        return None 
    
    i, j = best_pair
    dpx, dpy = profiles[i], profiles[j]
    motif = [dpx,dpy]
    #motif = [round((x + y) / 2, 2) for x, y in zip(dpx, dpy)] # choose average to represent motif
    return {
        "motif": motif,
        "pair_idx": (i, j),
        "score": round(min_distance, 2)
    }

def calculate_all_motifs(profiles, gamma, tau):
    motifs = []
    profiles = [tuple(dp) for dp in profiles]
    remaining_profiles = profiles.copy()
    count = 1
    while True:
        result = calculate_single_motif(remaining_profiles, gamma)
        print(f"all motifs : {result}")
        if result is None:
            break 
        
        motif_info = {
            "idx" : count,
            "motif" : result["motif"],
            "pair_idx" : result["pair_idx"],
            "score" : result["score"]
            }
        motifs.append(motif_info)
        i, j = motif_info["pair_idx"]
        dpx = remaining_profiles[i]
        dpy = remaining_profiles[j]
        min_distance = motif_info["score"]
        
        to_remove = set()
        to_remove.add(i)
        to_remove.add(j)
        
        for idx, dp in enumerate(remaining_profiles):
            if idx in to_remove:
                continue
            if (calculate_f_jk(dpx, dp) <= min_distance + tau or
                calculate_f_jk(dpy, dp) <= min_distance + tau):
                to_remove.add(idx)
        
        print(f"To remove : {to_remove}")
        remaining_profiles = [dp for idx, dp in enumerate(remaining_profiles) if idx not in to_remove]
    
    return motifs


In [95]:
profiles = [
    (100, 120, 115),    # dp0
    (100, 120, 110),    # dp1
    (110, 110, 110),    # dp2
    (140, 60, 194),     # dp3
    (150, 50, 200)      # dp4
]

gamma = 20
tau = 10

results = calculate_all_motifs(profiles, gamma, tau)

for i, motif_info in enumerate(results):
    print(f"Motif {i+1}:")
    print(f"  Represent motif: {motif_info['motif']}")
    print(f"  ID of x_t : {motif_info['pair_idx']}")
    print(f"  Score : {motif_info['score']}\n")

all motifs : {'motif': [(100, 120, 115), (100, 120, 110)], 'pair_idx': (0, 1), 'score': 5.0}
To remove : {0, 1, 2}
all motifs : {'motif': [(140, 60, 194), (150, 50, 200)], 'pair_idx': (0, 1), 'score': 15.36}
To remove : {0, 1}
all motifs : None
Motif 1:
  Represent motif: [(100, 120, 115), (100, 120, 110)]
  ID of x_t : (0, 1)
  Score : 5.0

Motif 2:
  Represent motif: [(140, 60, 194), (150, 50, 200)]
  ID of x_t : (0, 1)
  Score : 15.36



In [96]:
def calculate_label_dpk(motifs,dpk,tau):
    min_score = float('inf')
    best_pair = None

    for i, motif_info in enumerate(motifs):
        for j, dp in enumerate(motif_info['motif']):
            score = calculate_f_jk(dpk, dp)
            if score < min_score and score <= (motif_info['score'] + tau):
                min_score = score
                best_pair = (i, j)
    
    if best_pair is None:
        return None 
    
    i, j = best_pair
    print(motifs[i])
    dpk_info = {
        "dpk" : dpk,
        "label" : motifs[i]["idx"],
        "from" : motifs[i]['motif'][j]
        }
    
    return dpk_info
    

In [97]:
dpk = (130, 65, 190)

label = calculate_label_dpk(results,dpk,tau)

{'idx': 1, 'motif': [(140, 60, 194), (150, 50, 200)], 'pair_idx': (0, 1), 'score': 15.36}


In [98]:
print(label)

{'dpk': (130, 65, 190), 'label': 1, 'from': (140, 60, 194)}
